In [ ]:
#! pip2 install opencv-python

In [2]:
import cv2, glob, numpy, os
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
scale=300
image_size = 540
source_dir='/media/carlos_bologna/WD_NTFS/Diabetic_Retinopathy/original_images/'
dest_dir='data/'

In [4]:
def scaleRadius(img,scale):
    x=img[img.shape[0]/2,:,:].sum(1)
    r=(x>x.mean()/10).sum() / 2
    s=scale * 1.0 / r
    return cv2.resize(img,(0,0),fx=s,fy=s)
